In [82]:
import keras
import numpy as np
import pickle

## Inference

In [83]:
latent_dim = 256 

In [84]:
pickle_file = './data/index.pkl'

with open(pickle_file, 'rb') as file:
    loaded_data = pickle.load(file)

input_token_index = loaded_data['input_token_index']
target_token_index = loaded_data['target_token_index']

pickle_file = './data/data.pkl'

with open(pickle_file, 'rb') as file:
    loaded_data = pickle.load(file)

encoder_input_data = loaded_data['encoder_input_data']
decoder_input_data = loaded_data['decoder_input_data']
decoder_target_data = loaded_data['decoder_target_data']

print("Data loaded from pickle files.")

Data loaded from pickle files.


In [85]:
encoder_input_data = encoder_input_data[40000:]
decoder_input_data = decoder_input_data[40000:]
decoder_target_data = decoder_target_data[40000:]

In [86]:
num_decoder_tokens = len(target_token_index)
num_encoder_tokens = len(input_token_index)
max_decoder_seq_length = decoder_input_data.shape[1]
max_encoder_seq_length = encoder_input_data.shape[1]

In [ ]:
# load the model trained
model = keras.models.load_model("./models/s2s_model.keras")

# encoder model
encoder_inputs = model.input[0] 
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

# decoder model (unique names for each input layer to avoid name conflicts)
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens), name="decoder_input")
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="decoder_state_input_h")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="decoder_state_input_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# LSTM and Dense layers from the original model
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

# decoder model
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # first character of target sequence is the start character
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # either hit max length or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]

    return decoded_sentence


## Loss and Accuracy on test set

### V4 :
- Accuracy: 0.81, Test Accuracy: 0.82
- Loss: 0.61, Test Loss: 0.60

In [88]:
test_loss, test_accuracy = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

c:\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_4', 'input_layer_5']. Received: the structure of inputs=('*', '*')
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 92ms/step - accuracy: 0.8347 - loss: 0.5316
Test Loss: 0.5305841565132141, Test Accuracy: 0.835210919380188


## Test with custom Strings

In [89]:
def encode_string(input_string):
  encoder_input_data = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
  )
  t = 0
  for char in input_string:
    encoder_input_data[0, t, input_token_index[char]] = 1.0
    t += 1
  encoder_input_data[0, t + 1 :, input_token_index[" "]] = 1.0

  return encoder_input_data

In [99]:
string_to_translate = "i can read."

input_seq = encode_string(string_to_translate)
decoded_sentence = decode_sequence(input_seq)
print("Input sentence:", string_to_translate)
print("Decoded sentence:", decoded_sentence)

Input sentence: i can read.
Decoded sentence: je peux te marier.

